In [15]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import the Python CRUD Module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# update with your username and password and CRUD Python module name. 
#(AnimalShelter can take additional parameters specifying host/port/database/collection)

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

df['age_upon_outcome']

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center([
        html.A(
            html.Img(src="/assets/Grazioso Salvare Logo.png", alt="Grazioso Salvare Logo", height="200px"),
            href="https://www.snhu.edu"
        ),
        html.B(html.H1('AAC Database Dashboard')),
        "CS340-Lorenz-Michael"]
    ),
    html.Hr(),
    html.Div([
        # Entries per page selector
        "Entries per page: ",
        dcc.Dropdown([10, 15, 25, 50], 15, id="num_entries_dropdown", searchable=False, clearable=False),
        # Checkbox for species, allows filtering by multiple species
        "Species: ",
        dcc.Dropdown(df.animal_type.unique(), id="species_dropdown", placeholder="Select species", multi=True),
        # Checkbox for outcome types, allows filtering by multiple outcomes
        "Outcome Types: ",
        dcc.Dropdown(df.outcome_type.unique(), id="outcome_dropdown", placeholder="Select outcomes", multi=True),
        # Checkbox for dog occuptation type, allows filtering by 1 occupation to display preferred breeds/ages
        "Preferred Dog Occupation: ",
        dcc.RadioItems(
            {'WR':'Water Rescue', # value = 'WR', user sees 'Water Rescue'
             'MWR':'Mountain or Wilderness Rescue', 
             'DR':'Disaster Rescue or Individual Tracking'},
            id='dog_radio',
            inline=True), # each option displayed on one line
        html.Button('Reset Filters', id='button_reset_filters')
    ]),    
        dash_table.DataTable(
        id='datatable-id',
        columns=[ # Specify which columns to show with readable names
            # columns not shown: age_upon_outcome_in_weeks,monthyear, 
            {"name": "#", "id": "rec_num"},
            {"name": "ID", "id": "animal_id"},
            {"name": "Name", "id": "name"},
            {"name": "Species", "id": "animal_type"},
            {"name": "Breed", "id": "breed"},
            {"name": "Color", "id": "color"},
            {"name": "Sex", "id": "sex_upon_outcome"},
            {"name": "Date of Birth", "id": "date_of_birth"},
            {"name": "Outcome Age", "id": "age_upon_outcome"},
            {"name": "Outcome", "id": "outcome_type"},
            {"name": "Sub-Outcome", "id": "outcome_subtype"},
            {"name": "Outcome Date", "id": "datetime"},
            {"name": "Latitude", "id": "location_lat"},
            {"name": "Longitude", "id": "location_long"}
            #{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #features to make datatable interactive/user-friendly
        filter_action="native", # enable filtering
        sort_action="native", # enable sorting
        sort_mode="single",
        row_selectable = "single", # enable single row selection
        selected_rows = [0],
        page_size = 20 # default number of rows shown per page
    ),
    html.Br(),
    html.Hr(),
    # Interactive Leaflet Components
    html.Div([
        # Geographic Map
        html.Div([ 
            html.H4("Geographic Location of Selected Entry"),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )],
            style={'flex': '1', 'margin-left': '140px'}
        ),
        # Breed Pie Chart
        html.Div([ 
            html.H4("Analysis of Breeds"),
            dcc.Graph(id="graph_breeds")], 
            style={'flex':'1', 'margin-right': '150px'}
        )],
        # Flex styling to allow side-by-side display
        style= {
            'display': 'flex',
            'flex-direction': 'row',
        }
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
""" returns a dictionary that can be used to query for specified dogs based on the value
occupation_value is the occupation code string that retrieves queries for the following values:
'WR': water rescue
    breeds: lab retriever mix, chesapeake bay retriever, newfoundland
    sex: intact Female
    age: 25-156 weeks
'MWR': mountain or wilderness rescue
    breeds: german shepherd, alaskan malamute, old english sheepdog, siberian husky, rottweiler
    sex: intact male
    age: 25-156 weeks
'DR': disaster rescue or individual tracking
    breeds: doberman pinscher, german shephard, golden retriever, bloodhound, rottweiler
    sex: intact male
    age: 20-300 weeks
"""
def get_occupation_query(occupation_value):
    queryDict = {}
    queryDict['animal_type'] = 'Dog'
    # Water Rescue
    if (occupation_value == 'WR'):
        queryDict['breed'] = {'$in' : ['Labrador Retriever Mix','Chesa Bay Retr Mix',
                                       'Newfoundland']}
        queryDict['sex_upon_outcome'] = 'Intact Female'
        queryDict['age_upon_outcome_in_weeks'] = {'$gte':25, '$lte':156}
    # Mountain or Wilderness Rescue
    elif (occupation_value == 'MWR'):
        queryDict['breed'] = {'$in' : ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                       'Siberian Husky', 'Rottweiler']}
        queryDict['sex_upon_outcome'] = 'Intact Male'
        queryDict['age_upon_outcome_in_weeks'] = {'$gte':25, '$lte':156}
    # Disaster Rescue or Individual Tracking
    elif (occupation_value == 'DR'):
        queryDict['breed'] = {'$in' : ['German Shepherd', 'Doberman Pinsch', 'Golden Retriever',
                                       'Bloodhound', 'Rottweiler']}
        queryDict['sex_upon_outcome'] = 'Intact Male'
        queryDict['age_upon_outcome_in_weeks'] = {'$gte':20, '$lte':300}
    return queryDict
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

# This callback will adjust the number of rows shown per page selected in the dropdown
@app.callback(
    Output('datatable-id', 'page_size'),
    Input('num_entries_dropdown', 'value'))
def update_entries_per_page(page_size):
    return page_size

# This callback updates data table based on multiple filter selections. Including:
# - filter by species (multiple selections)
# - filter by outcomes (multiple selections)
# - filter by dog breeds/sex/ages preferred for specified occupations (single selection)
@app.callback(
    Output('datatable-id', 'data'),
    [Input('species_dropdown', 'value'),# value is array of items selected
     Input('outcome_dropdown', 'value'), # value is array of items selected
     Input('dog_radio', 'value')] # value is single selected value
)
def update_entries_shown(species, outcomes, occupation):
    queryDict = {}

    if occupation: # include entries that match the occupation specified
        queryDict = get_occupation_query(occupation)
    if species: # include all selected species in query
        queryDict['animal_type'] = {'$in' : species }
    if outcomes: # include all selected outcomes in query
        queryDict['outcome_type'] = {'$in' : outcomes }

    df = pd.DataFrame.from_records(shelter.read(queryDict))
    # cleanup _id field
    
    df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records') 

# This callback resets the filter selection when the reset button is pressed
@app.callback(
    [Output('species_dropdown', 'value'),
     Output('outcome_dropdown', 'value'),
     Output('dog_radio', 'value')],
    Input('button_reset_filters', 'n_clicks')
)
def reset_filters(n_clicks):
    return None, None, None # resets the selected value in each component

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):   
    # return empty map if viewData not provided
    if not viewData:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
    dl.Map(style={'width': '600px', 'height': '500px'},
       center=[30.75,-97.48], zoom=10, children=[
       dl.TileLayer(id="base-layer-id"),
       # Marker with tool tip and popup
       dl.Marker(position=[dff.loc[row,'location_lat'],dff.loc[row,'location_long']],
          children=[
          dl.Tooltip(dff.loc[row,'breed']), # label tooltip with breed
          dl.Popup([ # show animal name when marker is clicked
             html.H1("Animal Name"),
            html.P(dff.loc[row,'name'])
         ])
      ])
    ])
    ]

# This callback will update the pie chart with all of the available values visible in the "Breeds" column of the
# data table.
@app.callback(
    Output("graph_breeds", "figure"),
    Input("datatable-id", "derived_virtual_data"))
def update_graph(viewData):
    # return empty chart if viewData not provided
    if not viewData:
        return px.pie()
    dff = pd.DataFrame.from_dict(viewData)
    
    breed_counts = dff['breed'].value_counts(); # creates groups of breed : breed count
    
    threshold = breed_counts.sum() * .015 # filter out breeds that make up less than 1.5% to an "other" category
    
    breed_filtered = breed_counts[breed_counts >= threshold] # show breeds with counts at least 1.5% of total
    num_other = breed_counts[breed_counts < threshold].sum() # number of breeds less than threshold
    
    # add breeds less than 1.5% and add to a "other" category if needed
    if (num_other > 0):
        breed_filtered = pd.concat([breed_filtered, pd.Series({'Other': num_other})])
    
    # return pie chart with slices showing proportion of each breed, combining breeds under 1.5% into 'Other'
    return px.pie(
        dff, 
        values=breed_filtered.values, 
        names=breed_filtered.index,
        title='Breeds')
    

app.run_server(debug=True)

dash: 2.8.1
dash_leaflet: 0.1.23
pandas: 1.4.2
numpy: 1.21.5
matplotlib: 3.5.1
plotly: 5.6.0
pymongo: 3.12.0
Dash app running on http://127.0.0.1:14911/
